<a href="https://colab.research.google.com/github/EmirofThing/ME_489/blob/main/HW5_Group21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%writefile kmeans.cu

#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <cuda_runtime.h>
#include <time.h>

int Nd, Nc, Np;
double TOL;

#define BUFSIZE 512
#define MAX(x, y) (((x) > (y)) ? (x) : (y))
#define MIN(x, y) (((x) < (y)) ? (x) : (y))

#define MAX_ITER 10000

#define cudaCheckError() { \
    cudaError_t e = cudaGetLastError(); \
    if(e != cudaSuccess) { \
        printf("Cuda failure %s:%d: '%s'\n", __FILE__, __LINE__, cudaGetErrorString(e)); \
        exit(EXIT_FAILURE); \
    } \
}

__device__ double atomicAddDouble(double* address, double val) {
    unsigned long long int* address_as_ull = (unsigned long long int*)address;
    unsigned long long int old = *address_as_ull, assumed;
    do {
        assumed = old;
        old = atomicCAS(address_as_ull, assumed,
                        __double_as_longlong(val + __longlong_as_double(assumed)));
    } while (assumed != old);
    return __longlong_as_double(old);
}

__global__ void calculateDistances(double *data, double *centroids, double *distances, int Nd, int Nc, int Np) {
    extern __shared__ double s_centroids[];
    int tid = blockIdx.x * blockDim.x + threadIdx.x;

    for (int n = threadIdx.x; n < Nc * Nd; n += blockDim.x) {
        s_centroids[n] = centroids[n];
    }
    __syncthreads();

    if (tid < Np) {
        for (int n = 0; n < Nc; n++) {
            double sum = 0.0;
            for (int dim = 0; dim < Nd; dim++) {
                double diff = data[tid * Nd + dim] - s_centroids[n * Nd + dim];
                sum += diff * diff;
            }
            distances[tid * Nc + n] = sum;
        }
    }
}

__global__ void assignPoints(double *distances, int *Ci, int *Ck, int Nc, int Np) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;

    if (tid < Np) {
        double min_distance = INFINITY;
        int cluster_index = 0;

        for (int n = 0; n < Nc; n++) {
            if (distances[tid * Nc + n] < min_distance) {
                min_distance = distances[tid * Nc + n];
                cluster_index = n;
            }
        }

        atomicAdd(&Ck[cluster_index], 1);
        Ci[tid] = cluster_index;
    }
}

__global__ void updateCentroids(double *data, int *Ci, int *Ck, double *centroids, int Nd, int Nc, int Np) {
    extern __shared__ double s_data[];

    int tid = threadIdx.x;
    int cluster_index = blockIdx.x;

    for (int dim = 0; dim < Nd; dim++) {
        s_data[tid * Nd + dim] = 0.0;
    }
    __syncthreads();

    for (int i = tid; i < Np; i += blockDim.x) {
        if (Ci[i] == cluster_index) {
            for (int dim = 0; dim < Nd; dim++) {
                atomicAddDouble(&s_data[tid * Nd + dim], data[i * Nd + dim]);
            }
        }
    }
    __syncthreads();

    for (int stride = blockDim.x / 2; stride > 0; stride /= 2) {
        if (tid < stride) {
            for (int dim = 0; dim < Nd; dim++) {
                s_data[tid * Nd + dim] += s_data[(tid + stride) * Nd + dim];
            }
        }
        __syncthreads();
    }

    if (tid == 0) {
        for (int dim = 0; dim < Nd; dim++) {
            if (Ck[cluster_index] > 0) {
                centroids[cluster_index * Nd + dim] = s_data[dim] / Ck[cluster_index];
            }
        }
    }
}

double readInputFile(const char *fileName, const char* tag) {
    FILE *fp = fopen(fileName, "r");
    if (fp == NULL) {
        printf("Error opening the input file: %s\n", fileName);
        exit(EXIT_FAILURE);
    }

    int sk = 0;
    double result = -1;
    char buffer[BUFSIZE], fileTag[BUFSIZE];

    while (fgets(buffer, BUFSIZE, fp) != NULL) {
        sscanf(buffer, "%s", fileTag);
        if (strstr(fileTag, tag)) {
            fgets(buffer, BUFSIZE, fp);
            sscanf(buffer, "%lf", &result);
            sk++;
            break;
        }
    }

    fclose(fp);

    if (sk == 0) {
        printf("ERROR! Could not find the tag: [%s] in the file [%s]\n", tag, fileName);
        exit(EXIT_FAILURE);
    }
    return result;
}

void readDataFile(const char *fileName, double *data) {
    FILE *fp = fopen(fileName, "r");
    if (fp == NULL) {
        printf("Error opening the input file: %s\n", fileName);
        exit(EXIT_FAILURE);
    }

    int sk = 0;
    char buffer[BUFSIZE];

    int shift = Nd;
    while (fgets(buffer, BUFSIZE, fp) != NULL) {
        if (Nd == 2)
            sscanf(buffer, "%lf %lf", &data[sk * shift + 0], &data[sk * shift + 1]);
        if (Nd == 3)
            sscanf(buffer, "%lf %lf %lf", &data[sk * shift + 0], &data[sk * shift + 1], &data[sk * shift + 2]);
        if (Nd == 4)
            sscanf(buffer, "%lf %lf %lf %lf", &data[sk * shift + 0], &data[sk * shift + 1], &data[sk * shift + 2], &data[sk * shift + 3]);
        sk++;
    }

    fclose(fp);
}

void writeDataToFile(const char *fileName, double *data, int *Ci) {
    FILE *fp = fopen(fileName, "w");
    if (fp == NULL) {
        printf("Error opening the output file: %s\n", fileName);
        exit(EXIT_FAILURE);
    }

    for (int p = 0; p < Np; p++) {
        fprintf(fp, "%d %d ", p, Ci[p]);
        for (int dim = 0; dim < Nd; dim++) {
            fprintf(fp, "%.4f ", data[p * Nd + dim]);
        }
        fprintf(fp, "\n");
    }
    fclose(fp);
}

void writeCentroidToFile(const char *fileName, double *centroids) {
    FILE *fp = fopen(fileName, "w");
    if (fp == NULL) {
        printf("Error opening the output file: %s\n", fileName);
        exit(EXIT_FAILURE);
    }

    for (int n = 0; n < Nc; n++) {
        for (int dim = 0; dim < Nd; dim++) {
            fprintf(fp, "%.4f ", centroids[n * Nd + dim]);
        }
        fprintf(fp, "\n");
    }
    fclose(fp);
}

double calculateError(double *oldCentroids, double *newCentroids, int Nd, int Nc) {
    double sum = 0.0;
    for (int i = 0; i < Nc; i++) {
        for (int j = 0; j < Nd; j++) {
            sum += pow(newCentroids[i * Nd + j] - oldCentroids[i * Nd + j], 2);
        }
    }
    return sqrt(sum);
}

void kMeans(double *data, int *Ci, int *Ck, double *centroids) {
    double *d_data, *d_centroids, *d_distances;
    int *d_Ci, *d_Ck;

    size_t dataSize = Np * Nd * sizeof(double);
    size_t centroidSize = Nc * Nd * sizeof(double);
    size_t distanceSize = Np * Nc * sizeof(double);
    size_t CiSize = Np * sizeof(int);
    size_t CkSize = Nc * sizeof(int);

    cudaMalloc((void**)&d_data, dataSize);
    cudaMalloc((void**)&d_centroids, centroidSize);
    cudaMalloc((void**)&d_distances, distanceSize);
    cudaMalloc((void**)&d_Ci, CiSize);
    cudaMalloc((void**)&d_Ck, CkSize);

    cudaMemcpy(d_data, data, dataSize, cudaMemcpyHostToDevice);

    int threadsPerBlock = 256;
    int blocksPerGrid = (Np + threadsPerBlock - 1) / threadsPerBlock;

    // k-means++ başlangıç yöntemi
    int first_centroid_idx = rand() % Np;
    for (int dim = 0; dim < Nd; dim++) {
        centroids[0 * Nd + dim] = data[first_centroid_idx * Nd + dim];
    }

    double *h_distances = (double*)malloc(Np * sizeof(double));
    double *d_distances_sum;
    cudaMalloc((void**)&d_distances_sum, Np * sizeof(double));

    for (int k = 1; k < Nc; k++) {
        calculateDistances<<<blocksPerGrid, threadsPerBlock, Nc * Nd * sizeof(double)>>>(d_data, d_centroids, d_distances, Nd, k, Np);
        cudaMemcpy(h_distances, d_distances, Np * sizeof(double), cudaMemcpyDeviceToHost);

        double total_distance = 0;
        for (int i = 0; i < Np; i++) {
            double min_distance = h_distances[i * k];
            for (int j = 1; j < k; j++) {
                min_distance = MIN(min_distance, h_distances[i * k + j]);
            }
            h_distances[i] = min_distance;
            total_distance += h_distances[i];
        }

        double rand_val = ((double)rand() / RAND_MAX) * total_distance;
        int selected_idx = 0;
        for (int i = 0; i < Np; i++) {
            rand_val -= h_distances[i];
            if (rand_val <= 0) {
                selected_idx = i;
                break;
            }
        }

        for (int dim = 0; dim < Nd; dim++) {
            centroids[k * Nd + dim] = data[selected_idx * Nd + dim];
        }
        cudaMemcpy(d_centroids, centroids, (k + 1) * Nd * sizeof(double), cudaMemcpyHostToDevice);
    }

    free(h_distances);
    cudaFree(d_distances_sum);

    cudaMemcpy(d_centroids, centroids, centroidSize, cudaMemcpyHostToDevice);
    cudaMemcpy(d_Ci, Ci, CiSize, cudaMemcpyHostToDevice);
    cudaMemcpy(d_Ck, Ck, CkSize, cudaMemcpyHostToDevice);

    double *oldCentroids = (double *)malloc(centroidSize);
    double err = INFINITY;
    int sk = 0;

    cudaEvent_t start, stop;
    float elapsedTime;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    cudaEventRecord(start, 0);

    while (err > TOL && sk < MAX_ITER) {
        cudaMemset(d_Ck, 0, CkSize);

        cudaMemcpy(oldCentroids, d_centroids, centroidSize, cudaMemcpyDeviceToHost);

        calculateDistances<<<blocksPerGrid, threadsPerBlock, Nc * Nd * sizeof(double)>>>(d_data, d_centroids, d_distances, Nd, Nc, Np);
        cudaCheckError();

        assignPoints<<<blocksPerGrid, threadsPerBlock>>>(d_distances, d_Ci, d_Ck, Nc, Np);
        cudaCheckError();

        cudaMemset(d_centroids, 0, centroidSize);

        updateCentroids<<<Nc, threadsPerBlock, Nd * threadsPerBlock * sizeof(double)>>>(d_data, d_Ci, d_Ck, d_centroids, Nd, Nc, Np);
        cudaCheckError();

        cudaMemcpy(centroids, d_centroids, centroidSize, cudaMemcpyDeviceToHost);

        err = calculateError(oldCentroids, centroids, Nd, Nc);
        sk++;
    }

    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&elapsedTime, start, stop);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);

    printf("Execution Time: %f ms\n", elapsedTime);

    free(oldCentroids);

    cudaMemcpy(Ci, d_Ci, CiSize, cudaMemcpyDeviceToHost);
    cudaMemcpy(centroids, d_centroids, centroidSize, cudaMemcpyDeviceToHost);
    cudaMemcpy(Ck, d_Ck, CkSize, cudaMemcpyDeviceToHost);

    cudaFree(d_data);
    cudaFree(d_centroids);
    cudaFree(d_distances);
    cudaFree(d_Ci);
    cudaFree(d_Ck);
}

int main(int argc, char *argv[]) {
    if (argc != 3) {
        printf("Usage: ./kmeans input.dat data.dat\n");
        return -1;
    }

    Np = (int) readInputFile(argv[1], "NUMBER_OF_POINTS");
    Nc = (int) readInputFile(argv[1], "NUMBER_OF_CLUSTERS");
    Nd = (int) readInputFile(argv[1], "DATA_DIMENSION");
    TOL = readInputFile(argv[1], "TOLERANCE");

    double *data = (double*) malloc(Np * Nd * sizeof(double));
    int *Ci = (int *) calloc(Np, sizeof(int));
    int *Ck = (int *) calloc(Nc, sizeof(int));
    double *Cm = (double*) calloc(Nc * Nd, sizeof(double));

    readDataFile(argv[2], data);

    srand(time(NULL));
    kMeans(data, Ci, Ck, Cm);

    for (int n = 0; n < Nc; n++) {
        int Npoints = Ck[n];
        printf("(%d of %d) points are in the cluster %d with centroid( ", Npoints, Np, n);
        for (int dim = 0; dim < Nd; dim++) {
            printf("%f ", Cm[n * Nd + dim]);
        }
        printf(")\n");
    }

    writeDataToFile("output.dat", data, Ci);
    writeCentroidToFile("centroids.dat", Cm);

    free(data);
    free(Ci);
    free(Ck);
    free(Cm);

    return 0;
}


Overwriting kmeans.cu


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving circle_N4000.dat to circle_N4000.dat
Saving input.dat to input.dat


In [ ]:
# Derleme
!nvcc -o kmeans kmeans.cu

# Çalıştırma
!./kmeans input.dat circle_N4000.dat


In [ ]:
%%writefile kmeans.c

#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <string.h>
#include <time.h>

int Nd, Nc, Np;
double TOL;

#define BUFSIZE 512
#define MAX(x, y) (((x) > (y)) ? (x) : (y))
#define MIN(x, y) (((x) < (y)) ? (x) : (y))

// Define maximum number of iterations
#define MAX_ITER 10000

double readInputFile(char *fileName, char* tag) {
    FILE *fp = fopen(fileName, "r");
    if (fp == NULL) {
        printf("Error opening the input file\n");
        exit(EXIT_FAILURE);
    }

    int sk = 0; double result;
    char buffer[BUFSIZE], fileTag[BUFSIZE];

    while(fgets(buffer, BUFSIZE, fp) != NULL) {
        sscanf(buffer, "%s", fileTag);
        if(strstr(fileTag, tag)) {
            fgets(buffer, BUFSIZE, fp);
            sscanf(buffer, "%lf", &result);
            sk++;
            fclose(fp);
            return result;
        }
    }

    fclose(fp);
    if(sk == 0) {
        printf("ERROR! Could not find the tag: [%s] in the file [%s]\n", tag, fileName);
        exit(EXIT_FAILURE);
    }
    return -1; // hata durumunda
}

void readDataFile(char *fileName, double *data) {
    FILE *fp = fopen(fileName, "r");
    if (fp == NULL) {
        printf("Error opening the input file\n");
        exit(EXIT_FAILURE);
    }

    int sk = 0;
    char buffer[BUFSIZE], fileTag[BUFSIZE];

    int shift = Nd;
    while(fgets(buffer, BUFSIZE, fp) != NULL) {
        if(Nd == 2)
            sscanf(buffer, "%lf %lf", &data[sk * shift + 0], &data[sk * shift + 1]);
        if(Nd == 3)
            sscanf(buffer, "%lf %lf %lf", &data[sk * shift + 0], &data[sk * shift + 1], &data[sk * shift + 2]);
        if(Nd == 4)
            sscanf(buffer, "%lf %lf %lf %lf", &data[sk * shift + 0], &data[sk * shift + 1], &data[sk * shift + 2], &data[sk * shift + 3]);
        sk++;
    }

    fclose(fp);
}

void writeDataToFile(char *fileName, double *data, int *Ci) {
    FILE *fp = fopen(fileName, "w");
    if (fp == NULL) {
        printf("Error opening the output file\n");
        exit(EXIT_FAILURE);
    }

    for(int p = 0; p < Np; p++) {
        fprintf(fp, "%d %d ", p, Ci[p]);
        for(int dim = 0; dim < Nd; dim++) {
            fprintf(fp, "%.4f ", data[p * Nd + dim]);
        }
        fprintf(fp, "\n");
    }
    fclose(fp);
}

void writeCentroidToFile(char *fileName, double *Cm) {
    FILE *fp = fopen(fileName, "w");
    if (fp == NULL) {
        printf("Error opening the output file\n");
        exit(EXIT_FAILURE);
    }

    for(int n = 0; n < Nc; n++) {
        for(int dim = 0; dim < Nd; dim++) {
            fprintf(fp, "%.4f ", Cm[n * Nd + dim]);
        }
        fprintf(fp, "\n");
    }
    fclose(fp);
}

// Function to calculate Euclidean distance between two points
double distance(double *a, double *b) {
    double sum = 0.0;
    for(int dim = 0; dim < Nd; dim++) {
        sum += pow((a[dim] - b[dim]), 2);
    }
    return sqrt(sum);
}

// Function to calculate the error (Euclidean distance between old and new centroids)
double calculateError(double *oldCentroids, double *newCentroids, int Nd, int Nc) {
    double sum = 0.0;
    for (int i = 0; i < Nc; i++) {
        for (int j = 0; j < Nd; j++) {
            sum += pow(newCentroids[i * Nd + j] - oldCentroids[i * Nd + j], 2);
        }
    }
    return sqrt(sum);
}

// Function to assign each point to the nearest centroid
void assignPoints(double *data, int *Ci, int *Ck, double *Cm) {
    // Reset the number of points in the cluster
    for(int n = 0; n < Nc; n++) {
        Ck[n] = 0;
    }

    // Go over each point
    for (int p = 0; p < Np; p++) {
        double min_distance = INFINITY;
        int cluster_index = 0;

        for (int n = 0; n < Nc; n++) {
            double d = distance(&data[p * Nd], &Cm[n * Nd]);
            if (d < min_distance) {
                min_distance = d;
                cluster_index = n;
            }
        }

        Ck[cluster_index] += 1;
        Ci[p] = cluster_index;
    }
}

// Function to update centroids based on the mean of assigned points
double updateCentroids(double *data, int *Ci, int *Ck, double *Cm) {
    double *CmCopy = (double *)malloc(Nc * Nd * sizeof(double));
    memcpy(CmCopy, Cm, Nc * Nd * sizeof(double));

    for(int n = 0; n < Nc; n++) {
        for(int dim = 0; dim < Nd; dim++) {
            Cm[n * Nd + dim] = 0.0;
        }
    }

    for(int p = 0; p < Np; p++) {
        int cluster_index = Ci[p];

        for(int dim = 0; dim < Nd; dim++) {
            Cm[cluster_index * Nd + dim] += data[p * Nd + dim];
        }
    }

    double err = 1.E-12;
    for (int n = 0; n < Nc; n++) {
        for(int dim = 0; dim < Nd; dim++) {
            Cm[n * Nd + dim] = Cm[n * Nd + dim] / Ck[n];
            err = MAX(err, fabs(Cm[n * Nd + dim] - CmCopy[n * Nd + dim]));
        }
    }

    free(CmCopy);
    return err;
}

// Function to perform k-means clustering
void kMeans(double *data, int *Ci, int *Ck, double *Cm) {
    double *oldCentroids = (double *)malloc(Nc * Nd * sizeof(double));
    double err = INFINITY;
    int sk = 0;

    // Sabit başlangıç centroidleri
    for (int i = 0; i < Nc; i++) {
        for (int dim = 0; dim < Nd; dim++) {
            Cm[i * Nd + dim] = data[i * Nd + dim];  // İlk Nc veri noktasını kullanarak centroidleri başlat
        }
    }

    while (err > TOL && sk < MAX_ITER) {
        assignPoints(data, Ci, Ck, Cm);
        memcpy(oldCentroids, Cm, Nc * Nd * sizeof(double));
        err = updateCentroids(data, Ci, Ck, Cm);
        printf("\rIteration %d: %.12e", sk, err);
        sk++;
        fflush(stdout);
    }

    printf("\n");
    free(oldCentroids);
}

int main(int argc, char *argv[]) {
    if(argc != 3) {
        printf("Usage: ./kmeans input.dat data.dat\n");
        return -1;
    }

    // Başlangıç zamanını alın
    clock_t start = clock();

    // Read Number of Data Points
    Np = (int) readInputFile(argv[1], "NUMBER_OF_POINTS");
    // Read Number of clusters
    Nc = (int) readInputFile(argv[1], "NUMBER_OF_CLUSTERS");
    // Read Dimension of Data
    Nd = (int) readInputFile(argv[1], "DATA_DIMENSION");
    // Read Tolerance
    TOL = readInputFile(argv[1], "TOLERANCE");

    // Allocate data [x_i, y_i, z_i, ...]
    double *data = (double*) malloc(Np * Nd * sizeof(double));
    // Cluster id mapping every point to cluster
    int *Ci = (int *) calloc(Np, sizeof(int));
    // Number of data points in every cluster
    int *Ck = (int *) calloc(Nc, sizeof(int));
    // Centroid of every clusters
    double *Cm = (double*) calloc(Nc * Nd, sizeof(double));

    // Fill point data from file
    readDataFile(argv[2], data);

    // Perform k-means clustering
    kMeans(data, Ci, Ck, Cm);

    // Report Results
    for(int n = 0; n < Nc; n++) {
        int Npoints = Ck[n];
        printf("(%d of %d) points are in the cluster %d with centroid( ", Npoints, Np, n);
        for(int dim = 0; dim < Nd; dim++) {
            printf("%f ", Cm[n * Nd + dim]);
        }
        printf(")\n");
    }

    writeDataToFile("output.dat", data, Ci);
    writeCentroidToFile("centroids.dat", Cm);

    free(data);
    free(Ci);
    free(Ck);
    free(Cm);

    // Bitiş zamanını alın
    clock_t end = clock();

    // Geçen süreyi hesaplayın
    double elapsed_time = (double)(end - start) / CLOCKS_PER_SEC;
    printf("Execution Time: %.4f seconds\n", elapsed_time);

    return 0;
}


Overwriting kmeans.c


In [ ]:
!gcc kmeans.c -o kmeans -lm
!./kmeans input.dat circle_N4000.dat

cc1: fatal error: kmeans.c: No such file or directory
compilation terminated.
/bin/bash: line 1: ./kmeans: No such file or directory


In [ ]:
%%writefile kmeans_omp.c

#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <string.h>
#include <omp.h>
#include <time.h>

/* ************************************************************************** */
int Nd, Nc, Np;
double TOL;
int thread_count;

#define BUFSIZE 512

#define MAX(x, y) (((x) > (y)) ? (x) : (y))
#define MIN(x, y) (((x) < (y)) ? (x) : (y))

// Define maximum number of iterations
#define MAX_ITER 1000


/* ************************************************************************** */
double readInputFile(char *fileName, char* tag){
  FILE *fp = fopen(fileName, "r");
  // Error Check
  if (fp == NULL) {
    printf("Error opening the input file\n");
    exit(EXIT_FAILURE);
  }

  int sk = 0; double result;
  char buffer[BUFSIZE], fileTag[BUFSIZE];

  while(fgets(buffer, BUFSIZE, fp) != NULL){
    sscanf(buffer, "%s", fileTag);
    if(strstr(fileTag, tag)){
      fgets(buffer, BUFSIZE, fp);
      sscanf(buffer, "%lf", &result);
      sk++;
      return result;
    }
  }

  if(sk==0){
    printf("ERROR! Could not find the tag: [%s] in the file [%s]\n", tag, fileName);
    exit(EXIT_FAILURE);
  }
}


/* ************************************************************************** */
void readDataFile(char *fileName, double *data){
  FILE *fp = fopen(fileName, "r");
  if (fp == NULL) {
    printf("Error opening the input file\n");
    exit(EXIT_FAILURE);
  }

  int sk = 0;
  char buffer[BUFSIZE], fileTag[BUFSIZE];

  int shift = Nd;
  while(fgets(buffer, BUFSIZE, fp) != NULL){
    if(Nd==2)
      sscanf(buffer, "%lf %lf", &data[sk*shift + 0], &data[sk*shift+1]);
    if(Nd==3)
      sscanf(buffer, "%lf %lf %lf", &data[sk*shift + 0], &data[sk*shift+1], &data[sk*shift+2]);
    if(Nd==4)
      sscanf(buffer, "%lf %lf %lf %lf", &data[sk*shift+0],&data[sk*shift+1], &data[sk*shift+2], &data[sk*shift+3]);
    sk++;
  }
}

/* ************************************************************************** */
void writeDataFile(char *fileName, double *data, int *Ci){
  FILE *fp = fopen(fileName, "w");
  if (fp == NULL) {
    printf("Error opening the output file\n");
    exit(EXIT_FAILURE);
  }

  for(int p=0; p<Np; p++){
    fprintf(fp, "%d %d ", p, Ci[p]);
    for(int dim=0; dim<Nd; dim++){
      fprintf(fp, "%.4f ", data[p*Nd + dim]);
    }
    fprintf(fp, "\n");
  }
  fclose(fp);
}

/* ************************************************************************** */
void writeCentroidFile(char *fileName, double *Cm){
  FILE *fp = fopen(fileName, "w");
  if (fp == NULL) {
    printf("Error opening the output file\n");
    exit(EXIT_FAILURE);
  }

  for(int n=0; n<Nc; n++){
    for(int dim=0; dim<Nd; dim++){
      fprintf(fp, "%.4f ", Cm[n*Nd + dim]);
    }
    fprintf(fp, "\n");
  }
  fclose(fp);
}

/*************************************************************************** */
// Function to calculate Euclidean distance between two points
double distance(double *a, double *b) {
  double sum = 0.0;
  for(int dim=0; dim < Nd; dim++){
    sum += pow((a[dim] - b[dim]), 2);
  }
  return sqrt(sum);
}


/*************************************************************************** */
// Function to assign each point to the nearest centroid
void assignPoints(double *data, int *Ci, int *Ck, double *Cm) {

  // Reset the number of points in the cluster
#pragma omp parallel for num_threads(thread_count)
  for(int n = 0; n < Nc; n++) {
    Ck[n] = 0;
  }


  // Go over each point
#pragma omp parallel for num_threads(thread_count) reduction(+:Ck[:Nc])
  for (int p = 0; p < Np; p++) {
    double min_distance = INFINITY;
    int cluster_index   = 0;

    for (int n = 0; n < Nc; n++){
      double d = distance(&data[p*Nd], &Cm[n*Nd]);
      if (d < min_distance) {
        min_distance  = d;
        cluster_index = n;
      }
    }
    Ck[cluster_index] +=1;
    Ci[p]              = cluster_index;
  }
}

/*************************************************************************** */
// Function to update centroids based on the mean of assigned points
double updateCentroids(double *data, int *Ci, int *Ck, double *Cm) {

double *CmCopy = (double *)malloc(Nc*Nd*sizeof(double));

#pragma omp parallel for num_threads(thread_count)	\
  default(none) shared(CmCopy, Cm, Nc, Nd) collapse(2)
  for(int n = 0; n < Nc; n++) {
    for(int dim = 0; dim<Nd; dim++){
      CmCopy[n*Nd + dim] =  Cm[n*Nd + dim];
    }
  }

#pragma omp parallel for num_threads(thread_count) collapse(2)
  for(int n = 0; n < Nc; n++) {
    for(int dim = 0; dim<Nd; dim++){
      Cm[n*Nd + dim] = 0.0;
    }
  }


#pragma omp parallel for num_threads(thread_count) reduction(+:Cm[:Nc*Nd])
  for(int p=0; p<Np; p++){
    // Get cluster of the point
    int cluster_index = Ci[p];
    double sum = 0.0;
    for(int dim = 0; dim<Nd; dim++){
      // reduction required here!
      Cm[cluster_index*Nd + dim] += data[p*Nd + dim];
    }
  }

double err = 1.E-12;
#pragma omp parallel for num_threads(thread_count) reduction(max:err)
  for (int n = 0; n < Nc; n++){
    for(int dim = 0; dim<Nd; dim++){
      Cm[n*Nd + dim] = Cm[n*Nd + dim]/Ck[n];
      err = MAX(err, fabs(Cm[n*Nd + dim] - CmCopy[n*Nd + dim]));
    }
  }

  free(CmCopy);
  return err;

}

/*************************************************************************** */
// Function to perform k-means clustering
void kMeans(double *data, int *Ci, int *Ck, double *Cm) {
    double *oldCentroids = (double *)malloc(Nc * Nd * sizeof(double));
    double err = INFINITY;
    int sk = 0;

    // Sabit başlangıç centroidleri
    for (int i = 0; i < Nc; i++) {
        for (int dim = 0; dim < Nd; dim++) {
            Cm[i * Nd + dim] = data[i * Nd + dim];  // İlk Nc veri noktasını kullanarak centroidleri başlat
        }
    }

    double tstart = omp_get_wtime();
    while (err > TOL && sk < MAX_ITER) {
        assignPoints(data, Ci, Ck, Cm);
        memcpy(oldCentroids, Cm, Nc * Nd * sizeof(double));
        err = updateCentroids(data, Ci, Ck, Cm);
        sk++;
    }
    double elapsed_time = omp_get_wtime() - tstart;
    printf("Execution Time: %.8f seconds\n", elapsed_time);

    free(oldCentroids);
}

/*************************************************************************** */

int main ( int argc, char *argv[] ){
  if(argc!=4){
    printf("Usage: %s input.dat data.dat num_threads\n", argv[0]);
    return -1;
  }

  // Read Number of Data Points
  Np = (int) readInputFile(argv[1], "NUMBER_OF_POINTS");
  // Read Number of clusters
  Nc = (int) readInputFile(argv[1], "NUMBER_OF_CLUSTERS");
  // Read Dimension of Data
  Nd = (int) readInputFile(argv[1], "DATA_DIMENSION");
  TOL = readInputFile(argv[1], "TOLERANCE");

  thread_count = atoi(argv[3]);  // num_threads argümanı alınıyor

  omp_set_num_threads(thread_count);

  // Allocate data [c_i, x_i, y_i, z_i, ...]
  double *data = (double*) malloc(Np*Nd*sizeof(double));
  // Cluster id for every point
  int      *Ci = (int *) calloc(Np, sizeof(int));
  // Number of data points in every cluster
  int      *Ck = (int *) calloc(Nc, sizeof(int));
  // Centroid of every clusters
  double   *Cm = (double*) calloc(Nc*Nd, sizeof(double));

  // Fill point data from file
  readDataFile(argv[2], data);

  // Perform k-means clustering
  kMeans(data, Ci, Ck, Cm);

  // Report Results
  for(int n=0; n<Nc; n++){
    int Npoints =Ck[n];
    printf("(%d of %d) points are in the cluster %d with centroid( ", Npoints, Np, n);
    for(int dim = 0; dim<Nd; dim++){
      printf("%f ", Cm[n*Nd + dim]);
    }
    printf(") \n");
  }

  writeDataFile("output.dat",    data, Ci);
  writeCentroidFile("centroids.dat", Cm);


  free(data);
  free(Ci);
  free(Ck);
  free(Cm);

  return 0;
}


Overwriting kmeans_omp.c


In [ ]:
!gcc -fopenmp -o kmeans_omp kmeans_omp.c -lm
!./kmeans_omp input.dat circle_N4000.dat 8

Execution Time: 0.02599607 seconds
(616 of 4000) points are in the cluster 0 with centroid( 1.478158 0.064673 ) 
(592 of 4000) points are in the cluster 1 with centroid( -0.200558 -1.523298 ) 
(1000 of 4000) points are in the cluster 2 with centroid( -1.488141 1.519959 ) 
(637 of 4000) points are in the cluster 3 with centroid( -1.927173 -1.425406 ) 
(673 of 4000) points are in the cluster 4 with centroid( 1.569268 1.912891 ) 
(482 of 4000) points are in the cluster 5 with centroid( 1.940390 -1.998295 ) 
